In [1]:
"""
找出RS在0.1，0.3，0.5，0.8，1时选出的算法名称
随机抽样算法在不同`{'dataset': 'spambase', 'metric': 'accuracy', 'dsr': 0.3} `
选出的前2个算法 ` {'models': ['adaboost', 'extra_trees']}`.
其他设置：

- 禁止降维（fsm=rf, fsr=1)
- 探索次数(30次)
 
"""
import os.path
import pandas as pd
from tshpo.lib_class import AnaHelper

# _file = AnaHelper.prepare_csv("c05_effect_of_sample_method_v2_original_20241025_0812.csv.gz", return_df=False)
_file = AnaHelper.prepare_csv("c05_effect_of_sample_method_v2_original_20250514_0409.csv.gz", return_df=False)
_generate_file = f"{_file}_acc.gz"
if not os.path.exists(_generate_file):
    _odf = pd.read_csv(_file)
    _odf['metric'] = "accuracy"
    _odf.to_csv(_generate_file)
df = pd.concat([
    AnaHelper.load_acc_select(os.path.basename(_file), alpha=0.005),
    AnaHelper.load_acc_select(os.path.basename(_generate_file), alpha=0.005)])
df

Parse configs


100%|██████████| 300/300 [00:09<00:00, 30.58it/s]


Parse configs


100%|██████████| 300/300 [00:09<00:00, 30.88it/s]


,top_n,acc,dataset,metric,fsm,fsr,dsm,dsr,n_exploration,hpo_opt_method,top_models_tshpo
0,1,1,pc4,roc_auc,RF,1,Halton,0.1,30,RS,[adaboost]
1,2,1,pc4,roc_auc,RF,1,Halton,0.1,30,RS,"[adaboost, mlp]"
2,3,1,pc4,roc_auc,RF,1,Halton,0.1,30,RS,"[adaboost, mlp, extra_trees]"
3,4,1,pc4,roc_auc,RF,1,Halton,0.1,30,RS,"[adaboost, mlp, extra_trees, sgd]"
4,5,1,pc4,roc_auc,RF,1,Halton,0.1,30,RS,"[adaboost, mlp, extra_trees, sgd, libsvm_svc]"
...,...,...,...,...,...,...,...,...,...,...,...
2395,4,1,phoneme,accuracy,RF,1,Stratified,1.0,30,RS,"[extra_trees, mlp, random_forest, adaboost]"
2396,5,1,phoneme,accuracy,RF,1,Stratified,1.0,30,RS,"[extra_trees, mlp, random_forest, adaboost, de..."
2397,6,1,phoneme,accuracy,RF,1,Stratified,1.0,30,RS,"[extra_trees, mlp, random_forest, adaboost, de..."
2398,7,1,phoneme,accuracy,RF,1,Stratified,1.0,30,RS,"[extra_trees, mlp, random_forest, adaboost, de..."


In [2]:
fdf = df[(df['top_n'] == 2)]
fdf

,top_n,acc,dataset,metric,fsm,fsr,dsm,dsr,n_exploration,hpo_opt_method,top_models_tshpo
1,2,1,pc4,roc_auc,RF,1,Halton,0.1,30,RS,"[adaboost, mlp]"
9,2,1,pc4,roc_auc,RF,1,Halton,0.3,30,RS,"[mlp, adaboost]"
17,2,1,pc4,roc_auc,RF,1,Halton,0.5,30,RS,"[mlp, adaboost]"
25,2,1,pc4,roc_auc,RF,1,Halton,0.8,30,RS,"[adaboost, mlp]"
33,2,1,pc4,roc_auc,RF,1,Halton,1.0,30,RS,"[mlp, extra_trees]"
...,...,...,...,...,...,...,...,...,...,...,...
2361,2,1,phoneme,accuracy,RF,1,Stratified,0.1,30,RS,"[mlp, extra_trees]"
2369,2,1,phoneme,accuracy,RF,1,Stratified,0.3,30,RS,"[mlp, extra_trees]"
2377,2,1,phoneme,accuracy,RF,1,Stratified,0.5,30,RS,"[mlp, extra_trees]"
2385,2,1,phoneme,accuracy,RF,1,Stratified,0.8,30,RS,"[mlp, extra_trees]"


In [4]:
fdf['dsm'].unique()

array(['Halton', 'LHS', 'RS', 'Sobol', 'Stratified'], dtype=object)

In [3]:
from pyutils.kvdb.kvdb_mysql import KVDBMySQL

db = KVDBMySQL(table_name="q11_tshpo_top2_on_dataset")
for k, v in fdf.iterrows():
    _key = {
        "dataset": v['dataset'],
        'metric': v['metric'],
        'dsr': v['dsr'],
        'dsm': v['dsm']
    }
    _val = {"models": v['top_models_tshpo']}
    print(_key, _val)
    db.add(_key, _val)

Init logging ...
{'dataset': 'pc4', 'metric': 'roc_auc', 'dsr': 0.1, 'dsm': 'Halton'} {'models': ['adaboost', 'mlp']}
{'dataset': 'pc4', 'metric': 'roc_auc', 'dsr': 0.3, 'dsm': 'Halton'} {'models': ['mlp', 'adaboost']}
{'dataset': 'pc4', 'metric': 'roc_auc', 'dsr': 0.5, 'dsm': 'Halton'} {'models': ['mlp', 'adaboost']}
{'dataset': 'pc4', 'metric': 'roc_auc', 'dsr': 0.8, 'dsm': 'Halton'} {'models': ['adaboost', 'mlp']}
{'dataset': 'pc4', 'metric': 'roc_auc', 'dsr': 1.0, 'dsm': 'Halton'} {'models': ['mlp', 'extra_trees']}
{'dataset': 'pc4', 'metric': 'roc_auc', 'dsr': 0.1, 'dsm': 'LHS'} {'models': ['adaboost', 'mlp']}
{'dataset': 'pc4', 'metric': 'roc_auc', 'dsr': 0.3, 'dsm': 'LHS'} {'models': ['adaboost', 'mlp']}
{'dataset': 'pc4', 'metric': 'roc_auc', 'dsr': 0.5, 'dsm': 'LHS'} {'models': ['adaboost', 'mlp']}
{'dataset': 'pc4', 'metric': 'roc_auc', 'dsr': 0.8, 'dsm': 'LHS'} {'models': ['adaboost', 'mlp']}
{'dataset': 'pc4', 'metric': 'roc_auc', 'dsr': 1.0, 'dsm': 'LHS'} {'models': ['mlp'